<a href="https://colab.research.google.com/github/sramponi86/FRM_Columbia_Mean_Variance_Optimization_and_Sharpe_Ratio/blob/main/FRM_Columbia_Portfolio_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd

# Load your historical return data (e.g., CSV with daily returns for multiple stocks)
data = pd.read_excel('Test.xlsx', index_col='date')  #, on_bad_lines='skip')
data

,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,0.046629,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
2016-01-05,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,-0.006857,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2016-01-06,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,-0.061485,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
2016-01-07,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,0.026476,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
2016-01-08,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,-0.002378,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,0.006579,0.001716,-0.006328,-0.003967,0.002010,0.025091,-0.006211,0.007981,0.018798,0.030441,...,0.001542,0.004773,0.001596,-0.016616,-0.001816,0.000519,0.002776,-0.008819,-0.000974,-0.004051
2017-12-22,0.002162,-0.004008,-0.000941,0.001589,-0.003352,-0.016057,0.004662,0.011067,0.009268,0.082888,...,0.003077,0.039678,0.012678,0.003671,-0.007754,-0.001817,-0.000832,0.004909,-0.006847,-0.010201
2017-12-26,0.001961,-0.002873,0.001763,-0.007968,-0.001680,-0.003243,-0.006221,-0.003150,-0.005550,0.033523,...,-0.004362,0.001524,0.003145,-0.012565,0.009569,0.002077,0.004844,0.006833,-0.002949,-0.002738


In [14]:
# Calculate mean and variance for each stock
mean_returns = data.mean()  # Sample mean for each asset
var_returns = data.var()    # Sample variance for each asset

In [15]:
# Calculate the global average mean return
global_mean = mean_returns.mean()

# Set alpha
alpha = 0.9

# Shrinkage estimation for the mean return of IBM
# Assuming "IBM" is one of the columns in the dataset
mu_ibm = alpha * mean_returns['IBM'] + (1 - alpha) * global_mean

# Print the result rounded to four decimal places
print(f"{mu_ibm:.4f}")

0.0003


In [16]:
cov_matrix = data.cov()

In [17]:
from scipy.optimize import minimize

# Set target return constraint (0.0005 or 0.05% per day)
target_return = 0.0005

# Define the number of assets
num_assets = len(mean_returns)

def portfolio_return(weights, mean_returns):
    return np.dot(weights, mean_returns)

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Define the objective function to minimize portfolio variance
def minimize_portfolio_risk(weights, cov_matrix):
    return portfolio_volatility(weights, cov_matrix)

# Define the constraints
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights = 1
    {'type': 'ineq', 'fun': lambda x: portfolio_return(x, mean_returns) - target_return}  # Expected return >= target_return
]

# Define the bounds (no short selling constraint)
bounds = [(0, 1) for _ in range(num_assets)]

# Initial guess for weights (evenly distributed)
initial_guess = np.array([1/num_assets] * num_assets)

# Optimize the portfolio
result = minimize(minimize_portfolio_risk, initial_guess,
                  args=(cov_matrix), method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized weights
optimized_weights = result.x

# Find the weight for IBM (assuming "IBM" is one of the column names in the data)
ibm_weight = optimized_weights[mean_returns.index.get_loc("IBM")]

# Print the result rounded to three decimal places
#print(f"{ibm_weight:.3f}")
ibm_weight

0.0796239288175809

In [20]:

max_leverage = 0.1

def portfolio_volatility_short_long(weights, cov_matrix):
  # Separate long and short positions
    long_positions = weights[:num_assets]
    short_positions = weights[num_assets:]

    # Calculate net weights
    net_weights = long_positions - short_positions
    return np.sqrt(np.dot(net_weights.T, np.dot(cov_matrix, net_weights)))

# Define the objective function to minimize portfolio variance
def minimize_portfolio_risk_short_long(weights, cov_matrix):
    return portfolio_volatility_short_long(weights, cov_matrix)

# Define constraints
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w[:num_assets] - w[num_assets:]) - 1},  # Sum of weights = 1
    {'type': 'ineq', 'fun': lambda w: portfolio_return((w[:num_assets] - w[num_assets:]), mean_returns) - target_return},  # Expected return >= target_return
    {'type': 'ineq', 'fun': lambda w: np.sum(w[num_assets:]) - max_leverage}  # Leverage constraint
]

# Define bounds (long and short positions must be >= 0)
bounds = [(0, 1) for _ in range(2 * num_assets)]

# Optimize the portfolio
result = minimize(minimize_portfolio_risk_short_long, initial_guess,
                  args=(cov_matrix), method='SLSQP', constraints=constraints, bounds=bounds)

# Get the optimized net weights
optimized_weights = result.x[:num_assets] - result.x[num_assets:]

# Find the weight for IBM (assuming "IBM" is one of the column names in the data)
ibm_weight = optimized_weights[mean_returns.index.get_loc("IBM")]

# Print the result rounded to three decimal places
#print(f"{ibm_weight:.3f}")
ibm_weight

ValueError: The number of bounds is not compatible with the length of `x0`.